In [2]:
# Mock VisionAgent (Image → Tags)

def fake_vision_agent(image):
    return ["mountains", "lake", "snow", "sunset"]

In [3]:
# Mock Similarity Search Results

def mock_similarity_search(tags):
    return [

        {
            "location": "Hallstatt, Austria",
            "score": 0.90,
            "description": "A charming alpine village by the lake, known for its affordability.",
            "image": "https://images.unsplash.com/photo-1605559424843-cb2f104b577b"
        },
        {
            "location": "Banff, Canada",
            "score": 0.88,
            "description": "A Canadian national park with lakes, glaciers, and snowy peaks.",
            "image": "https://images.unsplash.com/photo-1535914254981-b5012eebbd15"
        },
        {
            "location": "Interlaken, Switzerland",
            "score": 0.95,
            "description": "A scenic Swiss town with snowy mountains and turquoise lakes.",
            "image": "https://images.unsplash.com/photo-1578948958259-519de7a9f73d"
        }
    ]

In [1]:
import random

def build_prompt(tags, destination):

  # Safe fallback for unknown location
    location = destination['location']
    if not location or location.lower() == 'unknown':
        location_phrase = "this stunning landscape"
        alt_location_sentence = "This scenery is truly a hidden gem waiting to be explored."
    else:
        location_phrase = location
        alt_location_sentence = f"A similar travel destination could be '{location}'."

    similar_phrases = [
        f"A similar travel destination could be \"{destination['location']}\".",
        f"Another amazing place you might love is \"{destination['location']}\".",
        f"A great place to consider is \"{destination['location']}\"."
    ]
    chosen_phrase = random.choice(similar_phrases)

    return (
        f"""
        The user wants a destination that matches these tags: {tags}.

        The most visually similar destination is: {destination['location']}.
        Destination description: {destination['description']}.

        If the location is unknown, suggest a real or plausible location that matches the scenery based on the tags. If you can't, describe the scenery naturally without naming a place.

        Write a short explanation in exactly 8 sentences:
        1 Start with an sentence like : "What a stunning place!"
        2 Then say: "{chosen_phrase}"
        3 Then say: {alt_location_sentence}
        4 In the next sentence, briefly introduce the place (where it is, what it is).
        5 In the next sentence, highlight what makes it special.
        6 In the next sentence, describe what makes it visually or culturally unique.
        7 In the next sentence, say why it’s famous and why someone should visit.
        8 End with a friendly invitation to visit — keep it short and warm.
        Make this one paragraph. Do not include numbers. Keep the whole explanation in 200 tokens. Do not exceed this.

        IMPORTANT:
        - Make sure the description strictly relates to the tags and factual information about the location.
        - Avoid speculation or adding information not supported by the tags or description.
        - Maintain clarity and avoid generic statements.
        - Do not repeat phrases unnecessarily.
        """
    )

def build_feedback_prompt(tags, destination):

    feedback_prompt = f"""
    The user wants a destination that matches these tags: {tags}.

    The most visually similar destination is: {destination['location']}.
    Destination description: {destination['description']}.

    If the location is unknown, suggest a real or plausible location that matches the scenery based on the tags. If you can't, describe the scenery naturally without naming a place.

    Write a single, warm, descriptive message in **exactly 5 to 7 complete sentences**.
    Start with a phrase like “Oh, I see you want...” referencing the user’s interest.
    Mention the place name naturally and speak directly to the reader (do not use “we”).
    Describe what makes it special, what they can see or do there, and inspire them to visit.


    IMPORTANT:
        - Make sure the description strictly relates to the tags and factual information about the location.
        - Avoid speculation or adding information not supported by the tags or description.
        - Maintain clarity and avoid generic statements.
        - Do not repeat phrases unnecessarily.
        - Give one output suggestion only.
        - Do not include instructions — output only the final message.
        - Keep the total length under 200 tokens.

    Output only the final text.
    """
    return feedback_prompt


In [4]:
from PIL import Image
from google.colab import files

uploaded = files.upload()
file_name = list(uploaded.keys())[0]
img = Image.open(file_name)

Saving south-beach-jetty.jpg to south-beach-jetty.jpg


In [5]:
tags = fake_vision_agent(img)
# Match and display
results = mock_similarity_search(tags)
top_result = results[0]

In [14]:
from openai import OpenAI
# Hugging Face client setup
client = OpenAI(
    base_url="https://router.huggingface.co/v1",
    api_key='[YOUR_HUGGING_FACE_TOKEN]',
)

In [18]:
##### Image upload and search similar places

llm_prompt = build_prompt(tags, top_result)

completion = client.chat.completions.create(
  model="HuggingFaceH4/zephyr-7b-beta:featherless-ai",
  messages=[
    {"role": "system", "content": "You are a warm and friendly travel assistant."},
    {"role": "user", "content": llm_prompt}
  ],
  temperature=0.7,
  max_tokens=250  # adjust to fit your average message length
)


raw_output = completion.choices[0].message.content
clean_output = raw_output.replace("[/USER]", "").strip()
print(clean_output)

What a stunning place! Another travel destination that might interest you is Hallstatt, Austria. Tucked away in the Salzkammergut region, this charming alpine village is nestled between the majestic Dachstein massif and Hallstätter See, Austria's largest salt mine lake. With the snow-capped peaks of the Dachstein Krippenstein mountain range overlooking the turquoise waters, sunsets here are breathtaking. The colorful Alpine houses and the lake's stillness under the evening light create unique hues, a picturesque scene that will leave you enchanted. Hallstatt's ancient salt mines and 700-year-old salt mines are also a must-see. It's a UNESCO World Heritage site, a hub for winter sports, and a perfect blend of culture and nature. Visit for its stunning views, and the iconic beauty it offers.
